# 文本预处理
:label:`sec_text_preprocessing`

对于序列数据处理问题，我们在 :numref:`sec_sequence`中
评估了所需的统计工具和预测时面临的挑战。
这样的数据存在许多种形式，文本是最常见例子之一。
例如，一篇文章可以被简单地看作是一串单词序列，甚至是一串字符序列。
本节中，我们将解析文本的常见预处理步骤。
这些步骤通常包括：

1. 将文本作为字符串加载到内存中。
1. 将字符串拆分为词元（如单词和字符）。
1. 建立一个词表，将拆分的词元映射到数字索引。
1. 将文本转换为数字索引序列，方便模型操作。


In [25]:
import collections
import re
from d2l import torch as d2l

## 读取数据集

首先，我们从H.G.Well的[时光机器](https://www.gutenberg.org/ebooks/35)中加载文本。
这是一个相当小的语料库，只有30000多个单词，但足够我们小试牛刀，
而现实中的文档集合可能会包含数十亿个单词。
下面的函数(**将数据集读取到由多条文本行组成的列表中**)，其中每条文本行都是一个字符串。
为简单起见，我们在这里忽略了标点符号和字母大写。


In [26]:
#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',                   #下载文本
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save                                                    #读取文本
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:                              #打开文本
        lines = f.readlines()                                                       #读取所有行
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]      #忽略大小写，标点符号。

lines = read_time_machine()                                                         #读取所有行
print(f'# 文本总行数: {len(lines)}')                                                #输出所有行
print(lines[0])                                                                     #输出第1行
print(lines[10])                                                                    #输出第11行

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


## 词元化

下面的`tokenize`函数将文本行列表（`lines`）作为输入，
列表中的每个元素是一个文本序列（如一条文本行）。
[**每个文本序列又被拆分成一个词元列表**]，*词元*（token）是文本的基本单位。
最后，返回一个由词元列表组成的列表，其中的每个词元都是一个字符串（string）。


In [27]:
def tokenize(lines, token='word'):  #@save          #词元 返回词元列表组成的列表，词元默认是word
    """将文本行拆分为单词或字符词元"""
    if token == 'word':                             #如果词元是word（单词）
        return [line.split() for line in lines]     #根据空格进行拆分，返回单词
    elif token == 'char':
        return [list(line) for line in lines]       #如果词元是char（字符） 将这个整条字符串变为列表，每个单位存放一个字符
    else:                                           #其他类型则报错
        print('错误：未知词元类型：' + token)       

tokens = tokenize(lines)                            #传入文本条
for i in range(11):                                 #返回前10条文本
    print(tokens[i])                                #返回每行拆分后的列表

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


## 词表

词元的类型是字符串，而模型需要的输入是数字，因此这种类型不方便模型使用。
现在，让我们[**构建一个字典，通常也叫做*词表*（vocabulary），
用来将字符串类型的词元映射到从$0$开始的数字索引中**]。
我们先将训练集中的所有文档合并在一起，对它们的唯一词元进行统计，
得到的统计结果称之为*语料*（corpus）。
然后根据每个唯一词元的出现频率，为其分配一个数字索引。
很少出现的词元通常被移除，这可以降低复杂性。
另外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元“&lt;unk&gt;”。
我们可以选择增加一个列表，用于保存那些被保留的词元，
例如：填充词元（“&lt;pad&gt;”）；
序列开始词元（“&lt;bos&gt;”）；
序列结束词元（“&lt;eos&gt;”）。


In [34]:
class Vocab:  #@save                                                            #定义词汇表
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):          #初始化，tokens默认无，最小出现频率默认0。reserved_tokens 告诉你这是开头token，这是末尾token
        if tokens is None:                                                      #如果词表无
            tokens = []                                                         #则创建一个list保留词元
        if reserved_tokens is None:                                             #如果保留词表为无
            reserved_tokens = []                                                #创建一个list保留词元
        # 按出现频率排序
        counter = count_corpus(tokens)                                          #记录每个词的出现频率
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],         #sorted对所有可迭代的对象进行排序。 将字典counter按value值进行排序。key=lambda x: x[1] 为对前面的对象中的第二维数据（即value）的值进行排序。 
                                   reverse=True)                                #设置为降序，false为升序    （这是列表，存放的是元组）
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens                         #设置未知词元的内容,添加到索引为0的list表，后接保存的词表（这是列表） 给index返回token
        self.token_to_idx = {token: idx                                         #将索引，词元设置为字典，键值对（这是字典） 给token 返回index
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:                                   #遍历刚刚统计好的词元列表，包含了单词和频率
            if freq < min_freq:                                                 #如果出现的频率小于最小评率则退出
                break
            if token not in self.token_to_idx:                                  #如果词元没有出现在之前统计的字典中
                self.idx_to_token.append(token)                                 #那么就添加到索引（这是列表）
                self.token_to_idx[token] = len(self.idx_to_token) - 1           #然后，将对应的索引，添加到键值对上的值上（这是字典）

    def __len__(self):                                                          #返回词元索引列表的长度
        return len(self.idx_to_token)

    def __getitem__(self, tokens):                                              #传入token返回index
        if not isinstance(tokens, (list, tuple)):                               #如果tokens不是list类型或者是元组类型
            return self.token_to_idx.get(tokens, self.unk)                      #将词元添加到 未知索引
        return [self.__getitem__(token) for token in tokens]                    #循环遍历，添加到未知索引

    def to_tokens(self, indices):                                               #给一些下标，把对应的字符返回
        if not isinstance(indices, (list, tuple)):                              #如果不是list或者元素类型
            return self.idx_to_token[indices]                                   #返回对应索引的值
        return [self.idx_to_token[index] for index in indices]                  #循环递归返回对应索引的值

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):                                                      #词元频率
        return self._token_freqs                                                #返回词元频率

def count_corpus(tokens):  #@save                           #传入词元
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]   
        #上面的循环可以这么理解
        #for line in tokens:
        #   for token in line:
        #   这样的循环，获取所有的词元，然后将它变为列表         
    return collections.Counter(tokens)                  #对所有词元进行统计

我们首先使用时光机器数据集作为语料库来[**构建词表**]，然后打印前几个高频词元及其索引。


In [35]:
vocab = Vocab(tokens)                           #传入数据集，构造词表
print(list(vocab.token_to_idx.items())[:10])    #输出前10个高频词元

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


现在，我们可以(**将每一条文本行转换成一个数字索引列表**)。


In [39]:
for i in [0, 10]:                       #遍历0行和第10行
    print('文本:', tokens[i])           #输出文本内容，每一行为单位
    print('索引:', vocab[tokens[i]])    #输出索引（通过键值对获取）

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


## 整合所有功能

在使用上述函数时，我们[**将所有功能打包到`load_corpus_time_machine`函数中**]，
该函数返回`corpus`（词元索引列表）和`vocab`（时光机器语料库的词表）。
我们在这里所做的改变是：

1. 为了简化后面章节中的训练，我们使用字符（而不是单词）实现文本词元化；
1. 时光机器数据集中的每个文本行不一定是一个句子或一个段落，还可能是一个单词，因此返回的`corpus`仅处理为单个列表，而不是使用多词元列表构成的一个列表。


In [31]:
def load_corpus_time_machine(max_tokens=-1):  #@save                #加载时间机器数据集 默认max_tokens=-1，是后面对应索引到最后
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()                                     #读取文本所有行
    tokens = tokenize(lines, 'char')                                #读取所有词元，（以字符来分割）
    vocab = Vocab(tokens)                                           #传入数据集，进行预处理
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]    #理解为一长串的整数，每个整数对应一个token
    #对corpus的解释
    # for line in tokens:
    #     for token in line:
    #         填入对应词表索引
    if max_tokens > 0:                                              #如果最大词元列表>0
        corpus = corpus[:max_tokens]                                #索引到对应位置
    return corpus, vocab                                            #返回词元索引列表，语料库

corpus, vocab = load_corpus_time_machine()                          #加载数据集
len(corpus), len(vocab)                                             #查看词元索引列表长度，语料表长度

(170580, 28)

## 小结

* 文本是序列数据的一种最常见的形式之一。
* 为了对文本进行预处理，我们通常将文本拆分为词元，构建词表将词元字符串映射为数字索引，并将文本数据转换为词元索引以供模型操作。

## 练习

1. 词元化是一个关键的预处理步骤，它因语言而异。尝试找到另外三种常用的词元化文本的方法。
1. 在本节的实验中，将文本词元为单词和更改`Vocab`实例的`min_freq`参数。这对词表大小有何影响？


[Discussions](https://discuss.d2l.ai/t/2094)
